# TTC Subway Ridership

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import os

c:\Users\yuqih\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### 1. Import and Combine Data

In [16]:
# Find all ttc subway ridership data file from the path
filenames = [file for file in os.listdir("TTC_ridership") if 'ttc-subway-ridership' in file]
filenames

['ttc-subway-ridership-2014.csv',
 'ttc-subway-ridership-2015.csv',
 'ttc-subway-ridership-2016.csv',
 'ttc-subway-ridership-2018.csv',
 'ttc-subway-ridership-2022.csv']

In [19]:
# Import the TTC subway station name list
ttc_stations = pd.read_csv("TTC_ridership\TTC_stations.csv")
ttc_stations

,Station Name,Line
0,Finch,1
1,North York Centre,1
2,Sheppard-Yonge (1 YONGE-UNIVERSITY),1
3,York Mills,1
4,Lawrence,1
...,...,...
75,Lawrence East,3
76,Ellesmere,3
77,Midland,3
78,Scarborough Centre,3


In [4]:
# Separate the TTC subway stations to 4 subway lines
line1 = ttc_stations[ttc_stations["Line"] == 1]
line2 = ttc_stations[ttc_stations["Line"] == 2]
line3 = ttc_stations[ttc_stations["Line"] == 3]
line4 = ttc_stations[ttc_stations["Line"] == 4]

In [10]:
# Start counting
count = 0

# Run through all files to put ridership and year data
for filename in filenames:
    # Read the yearly ridership data
    ridership_yearly_data = pd.read_csv(filename, header = None)
    # Get the year from the file name
    year = filename.split(".")[0][-4:]
    # Separate the name from the data
    name = [ridership_yearly_data[0][i] for i in range(0, len(ridership_yearly_data), 2)]
    # Separate the corresponding ridership count from the data
    ridership = [ridership_yearly_data[0][i] for i in range(1, len(ridership_yearly_data)+1, 2)]
    
    if count == 0:
        # Create a new ridership dataframe with data
        ttc_subway_ridership = pd.DataFrame({"Station Name": name, str(year): ridership})
    else:
        # Merge two dataframe by add a column for that year
        ttc_subway_ridership = ttc_subway_ridership.merge(pd.DataFrame({"Station Name": name, str(year): ridership}), how = "left", on = "Station Name")
    count = count + 1

ttc_subway_ridership.head()
    

,Station Name,2014,2015,2016,2018,2022
0,Bloor-Yonge (1 YONGE-UNIVERSITY),"216,190","216,190","204,630","204,630","155,186"
1,Bathurst,"35,510","36,460","29,320","26,900","26,234"
2,College,"47,940","47,790","47,600","44,370","42,883"
3,Bay,"31,050","30,860","27,090","32,690","24,260"
4,Davisville,"23,040","25,330","24,300","25,990","13,973"


In [20]:
# Merge TTC station line to each station
ttc_subway_ridership = ttc_subway_ridership.merge(ttc_stations, how = "left", on = "Station Name")
ttc_subway_ridership.head(40)

,Station Name,2014,2015,2016,2018,2022,Line_x,Line_y
0,Bloor-Yonge (1 YONGE-UNIVERSITY),"216,190","216,190","204,630","204,630","155,186",1,1
1,Bathurst,"35,510","36,460","29,320","26,900","26,234",2,2
2,College,"47,940","47,790","47,600","44,370","42,883",1,1
3,Bay,"31,050","30,860","27,090","32,690","24,260",2,2
4,Davisville,"23,040","25,330","24,300","25,990","13,973",1,1
5,Bloor-Yonge (2 BLOOR-DANFORTH),"193,030","183,240","186,860","196,460","131,995",2,2
6,Downsview,"39,900","37,670","40,640",NaN,NaN,1,1
7,Broadview,"34,880","33,460","29,800","32,670","25,057",2,2
8,Dundas,"75,780","81,330","77,200","73,560","72,637",1,1
9,Castle Frank,"7,070","8,350","8,520","9,760","12,470",2,2


### 2. Find the busiest stations 

In [ ]:
# 